In [1]:
import pandas as pd
import openpyxl
import json

input_json_file = 'sample/en.json'
out_excel_file = 'sample/en_edited.xlsx'
translation_excel_file = 'sample/sample.xlsx'

In [2]:
with open(input_json_file) as f:
    js = json.load(f)

In [3]:
out_df = pd.DataFrame(list(js.items()),
                   columns=['Key', 'To be Translated'])

In [4]:
lang = 'Hindi'
english_col = 'English copy'

In [5]:
excel = pd.ExcelFile(translation_excel_file)
excelDf = pd.DataFrame([],
                   columns=['Key', lang])
allowed_values = ['x','y','z']

In [6]:
count = 0
for sheetName in excel.sheet_names:
    sheet = excel.parse(sheet_name = sheetName, header=1)
    FORMAT = [english_col,lang]
    for value in allowed_values:
        if value in sheet.columns:
            FORMAT.append(value)
    filteredSheet = sheet[FORMAT]
    sheet_no_na = filteredSheet.dropna(subset = [english_col, lang], inplace=False)
    sheet_new = sheet_no_na.rename(columns = {english_col: 'Key'}, inplace=False)
    count += sheet_new.count()
    excelDf = pd.concat([excelDf, sheet_new], axis=0)

In [7]:
excelDf['Key_lower'] = excelDf['Key'].str.lower()
out_df['Key_lower'] = out_df['Key'].str.lower()

In [8]:
excelDf_dropped = excelDf.drop_duplicates(subset=['Key'], keep='first')
out_df_dropped = out_df.drop_duplicates(subset=['Key'], keep='first')

In [9]:
print(out_df.count(), out_df_dropped.count())
print(excelDf.count(), excelDf_dropped.count())

Key                 562
To be Translated    562
Key_lower           562
dtype: int64 Key                 562
To be Translated    562
Key_lower           562
dtype: int64
Key          190
Hindi        190
x              1
Key_lower    190
dtype: int64 Key          173
Hindi        173
x              1
Key_lower    173
dtype: int64


In [27]:
out_df_dropped.count()

Key                 562
To be Translated    562
Key_lower           562
dtype: int64

In [25]:
df_diff = pd.merge(out_df_dropped, excelDf_dropped, on="Key_lower",how='left')

In [26]:
df_diff.count()

Key_x               564
To be Translated    564
Key_lower           564
Key_y                67
Hindi                67
x                     0
dtype: int64

In [23]:
df_diff = df_diff.drop_duplicates(subset=['Key_y'], keep='first')

In [24]:
df_diff.count()

Key_x               53
To be Translated    53
Key_lower           53
Key_y               52
Hindi               52
x                    0
dtype: int64

In [13]:
df_diff = df_diff[['Key_x', 'To be Translated', lang]]

In [14]:
df_diff_a = df_diff[df_diff[lang].isna()]

In [20]:
df_diff[df_diff[lang].isna()].count()

Key_x               468
To be Translated    468
Hindi                 0
dtype: int64

In [18]:
df_diff_a.count()

Key                 468
To be Translated    468
Hindi                 0
dtype: int64

In [16]:
df_diff_a = df_diff_a.rename(columns = {'Key_x': 'Key'}, inplace=False)

In [17]:
df_diff_a.to_excel(out_excel_file)